In [1]:
import nltk
import re
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import os
import string

In [2]:
def clean_lyrics(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('\n',' ')
    text = re.sub(' +',' ',text)
    return text

In [18]:
files = os.listdir('Lyrics\\')
data=[]
n=0
for file in files:
    path='Lyrics\\'+file
    song_id=file.split('_')[0]
    lyrics = open(path,'r',encoding='utf-8').read()
    lyrics=clean_lyrics(lyrics)
    data.append([song_id,lyrics])



KeyboardInterrupt: 

In [ ]:
clean_df =pd.DataFrame(data,columns=['song_id','lyrics'])
clean_df.to_csv('clean_lyrics.txt',sep='\t',index=False)

In [3]:
data = pd.read_csv('clean_lyrics.txt',sep='\t')
data.lyrics = data.lyrics.astype(str)

In [4]:
tagged_data = [TaggedDocument(song[1].split(),[str(song[0])]) for song in zip(list(data['song_id']),list(data['lyrics']))]

In [5]:
max_epochs = 100
vec_size = 50
alpha = 0.025
model = Doc2Vec(vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=10, dm =1)

In [6]:
model.build_vocab(tagged_data)
model.train(tagged_data,epochs=10,total_examples=model.corpus_count)

In [7]:
model.save('doc2vec_len50.model')

# Let's try it out

In [8]:
model = Doc2Vec.load('doc2vec_len50.model')
new_df = pd.read_csv('song_ids_full.txt',sep='\t')

In [9]:
pd.set_option('display.max_rows', 700)
new_df[new_df.artist_name.str.contains('Kanye')]

,song_title,song_id,url_extension,artist_name,artist_id
48757,.,4531995,/Kanye-west--lyrics,Kanye West,72
48758,"12,000 Acres",5780836,/Kanye-west-12000-acres-lyrics,Kanye West,72
48759,18 Years,3187721,/Kanye-west-18-years-lyrics,Kanye West,72
48760,2013 Jimmy Kimmel Interview (Part 1),235515,/Kanye-west-2013-jimmy-kimmel-interview-part-1...,Kanye West,72
48761,2013 Jimmy Kimmel Interview (Part 2),235522,/Kanye-west-2013-jimmy-kimmel-interview-part-2...,Kanye West,72
48762,2013 Yeezus Tour Dates,214545,/Kanye-west-2013-yeezus-tour-dates-annotated,Kanye West,72
48763,2014 Yeezus Tour Dates,333592,/Kanye-west-2014-yeezus-tour-dates-lyrics,Kanye West,72
48764,2015 BET Honors Visionary Award Acceptance Speech,717578,/Kanye-west-2015-bet-honors-visionary-award-ac...,Kanye West,72
48765,2016 MTV VMAs Speech,2846871,/Kanye-west-2016-mtv-vmas-speech-lyrics,Kanye West,72
48766,24,5955219,/Kanye-west-24-lyrics,Kanye West,72


In [18]:
song='2000'
best = model.docvecs.most_similar(song,topn=5)

best = [int(idx[0]) for idx in best]

In [21]:
model.docvecs.most_similar(song,topn=5)

[]

In [20]:
new_df[new_df.song_id.isin(best)]

,song_title,song_id,url_extension,artist_name,artist_id
12839,Twettle Bomb,2444644,/Bryson-tiller-twettle-bomb-lyrics,Bryson Tiller,264368
61597,Life Is Beautiful,4315361,/Locksmith-life-is-beautiful-annotated,Locksmith,724
65124,America the Beautiful,1043749,/Mariah-carey-america-the-beautiful-lyrics,Mariah Carey,1205
74682,Battle Cry,1559510,/Omen-battle-cry-lyrics,Omen,731
104699,Ramona Park is Yankee Stadium,3119152,/Vince-staples-ramona-park-is-yankee-stadium-l...,Vince Staples,941
